In [ ]:
# Simple Configuration
CSV_FOLDER = "/home/pyuser/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"

# Paths
DOWNLOAD_PATH = '/home/pyuser/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'

# Processing settings
TARGET_SIZE = (518, 518)
LUNG_THRESHOLD = 0.1  # Single threshold for lung detection
CROP_MARGIN = 40  # Margin around lungs for final crop

# V1.3 Style Overlay Settings
LUNG_FILL_OPACITY = 0.25  # Lung fill opacity (like V1.3)
LUNG_BORDER_OPACITY = 0.50  # Lung border opacity (like V1.3)

# Options
CONVERT = True
SAVE_MASKS = True

print("V1.4 Simplified configuration loaded!")
print(f"Target size: {TARGET_SIZE}")
print(f"Lung threshold: {LUNG_THRESHOLD}")
print("V1.3 style overlay enabled")


In [ ]:
# Core dependencies
import ArchiMedConnector.A3_Connector as A3_Conn
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
import cv2
import subprocess
import sys

print("Core dependencies loaded")

# Initialize ArchiMed connector
a3conn = A3_Conn.A3_Connector()


In [ ]:
# Simple segmentation model setup
segmentation_model = None
model_type = None

# Try TorchXRayVision first (best option)
try:
    import torchxrayvision as xrv
    import torch
    segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
    model_type = 'torchxray'
    print("✅ TorchXRayVision loaded")
except ImportError:
    print("⚠️ Installing TorchXRayVision...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torchxrayvision"])
        import torchxrayvision as xrv
        import torch
        segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
        model_type = 'torchxray'
        print("✅ TorchXRayVision installed and loaded")
    except:
        print("❌ TorchXRayVision unavailable, using fallback")
        model_type = 'fallback'

print(f"Segmentation method: {model_type}")


In [ ]:
def segment_lungs(image):
    """Simple lung segmentation"""
    if model_type == 'torchxray' and segmentation_model is not None:
        return segment_with_torchxray(image)
    else:
        return segment_with_fallback(image)

def segment_with_torchxray(image):
    """Segment using TorchXRayVision"""
    try:
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            image_gray = image
        
        # Normalize for TorchXRayVision
        image_norm = xrv.datasets.normalize(image_gray, 255)
        image_norm = image_norm[None, ...]
        
        # Resize to 512x512
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        # Run inference
        with torch.no_grad():
            output = segmentation_model(image_tensor)
        
        # Extract lung masks
        lung_targets = ['Left Lung', 'Right Lung']
        lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(segmentation_model.targets):
            if target in lung_targets:
                lung_mask += output[0, i].cpu().numpy()
        
        # Resize back to original size
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        
        # Create binary mask
        binary_mask = (lung_mask > LUNG_THRESHOLD).astype(np.uint8)
        
        # Clean up mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
        
        return binary_mask
        
    except Exception as e:
        print(f"TorchXRayVision segmentation failed: {e}")
        return segment_with_fallback(image)

def segment_with_fallback(image):
    """Fallback segmentation method"""
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image.copy()
    
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    
    # Otsu thresholding
    _, otsu_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphological cleanup
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
    mask_clean = cv2.morphologyEx(otsu_mask, cv2.MORPH_CLOSE, kernel)
    mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_OPEN, kernel)
    
    return (mask_clean > 0).astype(np.uint8)

print("Lung segmentation functions loaded")


In [ ]:
def create_v13_overlay(image, binary_mask, resize_crop_bounds):
    """Create V1.3 style overlay with cyan rectangle, green corners, and red lung filling"""
    overlay = image.copy()
    if len(overlay.shape) == 2:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
    resize_y_min, resize_x_min, resize_y_max, resize_x_max = resize_crop_bounds
    img_height, img_width = overlay.shape[:2]
    
    # 1. Darken areas OUTSIDE the resize crop (like V1.3)
    outside_resize_mask = np.ones((img_height, img_width), dtype=bool)
    outside_resize_mask[resize_y_min:resize_y_max, resize_x_min:resize_x_max] = False
    overlay[outside_resize_mask] = (overlay[outside_resize_mask] * 0.5).astype(np.uint8)
    
    # 2. Red lung visualization with dual opacity (exactly like V1.3)
    lung_areas = binary_mask > 0
    
    if np.any(lung_areas):
        print(f"📍 Debug: Found {np.sum(lung_areas)} lung pixels")
        
        # Step 1: Apply DARK RED FILL with 25% opacity (LUNG_FILL_OPACITY)
        lung_fill_colored = np.zeros_like(overlay)
        lung_fill_colored[lung_areas] = [0, 0, 180]  # Dark Red in BGR
        
        overlay[lung_areas] = cv2.addWeighted(
            overlay[lung_areas], 1.0 - LUNG_FILL_OPACITY, 
            lung_fill_colored[lung_areas], LUNG_FILL_OPACITY, 0
        )
        print(f"📍 Debug: Applied red fill with {LUNG_FILL_OPACITY} opacity")
        
        # Step 2: Apply DARK RED BORDER with 50% opacity (LUNG_BORDER_OPACITY)
        lung_mask_uint8 = (binary_mask * 255).astype(np.uint8)
        contours, _ = cv2.findContours(lung_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        border_mask = np.zeros_like(binary_mask, dtype=np.uint8)
        cv2.drawContours(border_mask, contours, -1, 1, thickness=4)  # 4px thick border
        
        border_areas = border_mask > 0
        if np.any(border_areas):
            lung_border_colored = np.zeros_like(overlay)
            lung_border_colored[border_areas] = [0, 0, 180]  # Dark Red in BGR
            
            overlay[border_areas] = cv2.addWeighted(
                overlay[border_areas], 1.0 - LUNG_BORDER_OPACITY, 
                lung_border_colored[border_areas], LUNG_BORDER_OPACITY, 0
            )
            print(f"📍 Debug: Applied red border with {LUNG_BORDER_OPACITY} opacity")
        else:
            print("📍 Debug: No border areas found")
    else:
        print("📍 Debug: No lung areas found")
    
    # 3. Green corner brackets for exact segmentation boundaries (exactly like V1.3)
    def draw_corner_brackets(img, x1, y1, x2, y2, color, thickness=3, length=40):
        """Draw corner brackets at the corners of a rectangle (exactly like V1.3)"""
        # Top-left corner
        cv2.line(img, (x1, y1), (x1 + length, y1), color, thickness)  # Horizontal
        cv2.line(img, (x1, y1), (x1, y1 + length), color, thickness)  # Vertical
        
        # Top-right corner
        cv2.line(img, (x2 - length, y1), (x2, y1), color, thickness)  # Horizontal
        cv2.line(img, (x2, y1), (x2, y1 + length), color, thickness)  # Vertical
        
        # Bottom-left corner
        cv2.line(img, (x1, y2 - length), (x1, y2), color, thickness)  # Vertical
        cv2.line(img, (x1, y2), (x1 + length, y2), color, thickness)  # Horizontal
        
        # Bottom-right corner
        cv2.line(img, (x2, y2 - length), (x2, y2), color, thickness)  # Vertical
        cv2.line(img, (x2 - length, y2), (x2, y2), color, thickness)  # Horizontal
    
    # Find actual segmentation boundaries (without padding) for green corners (exactly like V1.3)
    lung_coords = np.where(binary_mask > 0)
    if len(lung_coords[0]) > 0:
        actual_y_min = np.min(lung_coords[0])  # Top edge of lungs
        actual_y_max = np.max(lung_coords[0])  # Bottom edge of lungs  
        actual_x_min = np.min(lung_coords[1])  # Left edge of lungs
        actual_x_max = np.max(lung_coords[1])  # Right edge of lungs
        
        print(f"📍 Debug: Segmentation boundaries: ({actual_x_min},{actual_y_min}) to ({actual_x_max},{actual_y_max})")
        
        # Draw GREEN corner brackets at EXACT segmentation boundaries (no padding) - exactly like V1.3
        draw_corner_brackets(overlay, actual_x_min, actual_y_min, actual_x_max, actual_y_max, (0, 255, 0), 3, 40)
        print("📍 Debug: Drew green corner brackets with length=40")
    else:
        print("📍 Debug: No lung areas found for corner brackets")
    
    # 4. Draw CYAN resize crop rectangle (contains everything) - 1px contour line (exactly like V1.3)
    cv2.rectangle(overlay, (resize_x_min, resize_y_min), (resize_x_max, resize_y_max), (255, 255, 0), 1)
    print(f"📍 Debug: Drew cyan rectangle: ({resize_x_min},{resize_y_min}) to ({resize_x_max},{resize_y_max})")
    
    # 5. V1.3 style legend with BLACK BACKGROUND
    legend_height = 120
    legend_width = min(700, img_width - 20)  # Ensure legend fits within image bounds
    legend_y_start = img_height - legend_height - 10
    
    # Create black background with 75% opacity (exactly like V1.3)
    legend_background = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    legend_area = overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width]
    
    # Blend background with 75% opacity (25% transparency) - exactly like V1.3
    overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width] = cv2.addWeighted(
        legend_area, 0.25, legend_background, 0.75, 0
    )
    
    # Legend text with larger font (exactly like V1.3)
    text_y = legend_y_start + 25
    font_scale = 0.7  # Larger font
    font_thickness = 2
    
    cv2.putText(overlay, f"DARK RED = Lung segmentation (Fill: {int(LUNG_FILL_OPACITY*100)}%, Border: {int(LUNG_BORDER_OPACITY*100)}%)", 
               (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 180), font_thickness)
    
    cv2.putText(overlay, "GREEN = Segmentation corner brackets", 
               (20, text_y + 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
    
    cv2.putText(overlay, f"CYAN = Final resize crop {TARGET_SIZE[0]}x{TARGET_SIZE[1]}", 
               (20, text_y + 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 0), font_thickness)
    
    # Add feature information at bottom with smaller font (like V1.3)
    cv2.putText(overlay, f"Threshold: {LUNG_THRESHOLD} | Model: {model_type}", 
               (20, text_y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    return overlay

def process_image_with_segmentation(image_array, file_id):
    """Process image with lung segmentation and clear area cropping"""
    try:
        print(f"🫁 Segmenting lungs for {file_id}...")
        
        # Get lung segmentation
        binary_mask = segment_lungs(image_array)
        
        # Validate segmentation
        total_pixels = binary_mask.shape[0] * binary_mask.shape[1]
        lung_pixels = np.sum(binary_mask)
        lung_ratio = lung_pixels / total_pixels
        
        print(f"📊 Lung area: {lung_ratio:.3f} of image")
        
        # Quality check
        if lung_ratio < 0.005 or lung_ratio > 0.95:
            print(f"⚠️ Lung area outside valid range, using original")
            return image_array
        
        # Find lung boundaries
        coords = np.column_stack(np.where(binary_mask > 0))
        if len(coords) == 0:
            return image_array
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        # Calculate resize crop (clear area) - exactly like V1.3
        img_height, img_width = image_array.shape[:2]
        
        # Add padding around lung boundaries (like V1.3)
        safety_margin = 20
        y_min_padded = max(0, y_min - safety_margin)
        x_min_padded = max(0, x_min - safety_margin)
        y_max_padded = min(img_height, y_max + safety_margin)
        x_max_padded = min(img_width, x_max + safety_margin)
        
        # Find lung center for positioning the resize crop (like V1.3)
        lung_center_y = (y_min_padded + y_max_padded) // 2
        lung_center_x = (x_min_padded + x_max_padded) // 2
        
        # Calculate resize crop dimensions based on TARGET_SIZE (like V1.3)
        target_aspect_ratio = TARGET_SIZE[0] / TARGET_SIZE[1]  # width/height
        
        # Make resize crop large enough to contain the lung area, but respect aspect ratio
        lung_width = x_max_padded - x_min_padded
        lung_height = y_max_padded - y_min_padded
        
        # Calculate minimum size needed to contain lungs, then expand if needed (like V1.3)
        min_width = lung_width + 40  # Extra margin (like V1.3)
        min_height = lung_height + 40
        
        # Ensure aspect ratio is maintained (like V1.3)
        if min_width / min_height > target_aspect_ratio:
            # Width is limiting factor
            resize_width = min_width
            resize_height = int(resize_width / target_aspect_ratio)
        else:
            # Height is limiting factor  
            resize_height = min_height
            resize_width = int(resize_height * target_aspect_ratio)
        
        # Center resize crop on lung center, but keep within image bounds (like V1.3)
        resize_x_min = max(0, lung_center_x - resize_width // 2)
        resize_y_min = max(0, lung_center_y - resize_height // 2)
        resize_x_max = min(img_width, resize_x_min + resize_width)
        resize_y_max = min(img_height, resize_y_min + resize_height)
        
        # Adjust if we hit image boundaries (like V1.3)
        if resize_x_max == img_width:
            resize_x_min = img_width - resize_width
        if resize_y_max == img_height:
            resize_y_min = img_height - resize_height
            
        # Ensure non-negative coordinates (like V1.3)
        resize_x_min = max(0, resize_x_min)
        resize_y_min = max(0, resize_y_min)
        
        print(f"📐 Clear area crop: ({resize_y_min},{resize_x_min}) to ({resize_y_max},{resize_x_max})")
        
        # Crop to clear area
        if len(image_array.shape) == 3:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max, :]
        else:
            cropped = image_array[resize_y_min:resize_y_max, resize_x_min:resize_x_max]
        
        # Save masks if requested
        if SAVE_MASKS:
            os.makedirs(MASKS_PATH, exist_ok=True)
            
            # Save binary mask
            mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
            mask_image = (binary_mask * 255).astype(np.uint8)
            cv2.imwrite(mask_path, mask_image)
            
            # Save V1.3 style overlay
            overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
            resize_bounds = (resize_y_min, resize_x_min, resize_y_max, resize_x_max)
            overlay = create_v13_overlay(image_array, binary_mask, resize_bounds)
            cv2.imwrite(overlay_path, overlay)
            
            print(f"💾 Saved V1.3 style masks: {file_id}_mask.png, {file_id}_overlay.png")
        
        return cropped
        
    except Exception as e:
        print(f"⚠️ Processing failed for {file_id}: {e}")
        return image_array

print("Image processing with V1.3 overlay loaded")


In [ ]:
# Download images from ArchiMed
try:
    # Load CSV data
    user_info = a3conn.getUserInfos()
    print(f"🔐 ArchiMed connection: {user_info}")
    
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    print(f"✅ Loaded CSV with {len(df)} rows")
    
    # Find FileID column
    file_id_column = None
    for col in ['FileID', 'file_id', 'File_ID']:
        if col in df.columns:
            file_id_column = col
            break
    
    if file_id_column is None:
        raise ValueError("FileID column not found")
    
    # Get file IDs
    file_ids = df[file_id_column].dropna().unique()
    total_files = len(file_ids)
    
    print(f"🚀 Starting download of {total_files} files")
    print(f"Destination: {DOWNLOAD_PATH}")
    
    # Download files
    downloaded_files = []
    
    for i, file_id in enumerate(file_ids):
        progress = ((i + 1) / total_files) * 100
        file_id_str = str(file_id)
        print(f"⬇️ Downloading {file_id_str} ({progress:.1f}% - {i+1}/{total_files})")
        
        try:
            dicom_file_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
            os.makedirs(DOWNLOAD_PATH, exist_ok=True)
            
            if os.path.exists(dicom_file_path):
                print(f"File {file_id} already exists, skipping")
                downloaded_files.append(dicom_file_path)
                continue
            
            # Download file
            result = a3conn.downloadFile(
                int(file_id_str),
                asStream=False,
                destDir=DOWNLOAD_PATH,
                filename=f"{file_id_str}.dcm",
                inWorklist=False
            )
            
            if result and os.path.exists(dicom_file_path):
                downloaded_files.append(dicom_file_path)
                print(f"✅ Downloaded: {dicom_file_path}")
            else:
                print(f"⚠️ Download unclear for {file_id_str}")
                
        except Exception as e:
            print(f"⚠️ Failed to download {file_id_str}: {e}")
    
    print(f"✅ Downloaded {len(downloaded_files)} files successfully")
    
except Exception as e:
    print(f"❌ Download failed: {e}")
    downloaded_files = []


In [ ]:
# Convert DICOM files to images with segmentation
def convert_dicom_to_image(dicom_path, output_path, target_size=TARGET_SIZE):
    """Convert DICOM to image with lung segmentation and cropping"""
    try:
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        
        # Read DICOM
        dicom_data = pydicom.dcmread(dicom_path)
        image_array = dicom_data.pixel_array
        print(f"📁 Processing {file_id}: {image_array.shape}")
        
        # Handle MONOCHROME1
        if hasattr(dicom_data, 'PhotometricInterpretation'):
            if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                image_array = np.max(image_array) - image_array
        
        # Normalize to 0-255
        if image_array.max() > 255:
            image_array = ((image_array - image_array.min()) / 
                          (image_array.max() - image_array.min()) * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)
        
        # Apply lung segmentation and cropping
        processed_image = process_image_with_segmentation(image_array, file_id)
        
        # Convert to PIL
        if len(processed_image.shape) == 2:
            pil_image = Image.fromarray(processed_image, mode='L')
        else:
            pil_image = Image.fromarray(processed_image)
        
        # Resize to target size with aspect ratio preservation
        current_width, current_height = pil_image.size
        target_width, target_height = target_size
        
        current_ratio = current_width / current_height
        target_ratio = target_width / target_height
        
        print(f"📐 Current: {current_width}x{current_height} (ratio: {current_ratio:.3f})")
        print(f"📐 Target: {target_width}x{target_height} (ratio: {target_ratio:.3f})")
        
        # Center crop to match aspect ratio
        if current_ratio > target_ratio:
            # Crop width
            new_width = int(current_height * target_ratio)
            new_height = current_height
            left = (current_width - new_width) // 2
            top = 0
            right = left + new_width
            bottom = current_height
            print(f"📏 Cropping width: {current_width} → {new_width}")
        else:
            # Crop height
            new_width = current_width
            new_height = int(current_width / target_ratio)
            left = 0
            top = (current_height - new_height) // 2
            right = current_width
            bottom = top + new_height
            print(f"📏 Cropping height: {current_height} → {new_height}")
        
        # Apply crop and resize
        pil_image = pil_image.crop((left, top, right, bottom))
        pil_image = pil_image.resize(target_size, Image.Resampling.LANCZOS)
        print(f"✅ Final size: {target_width}x{target_height}")
        
        # Save image
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        pil_image.save(output_path)
        return True
        
    except Exception as e:
        print(f"❌ Failed to convert {dicom_path}: {e}")
        return False

# Convert downloaded files
if CONVERT and downloaded_files:
    print(f"🔄 Converting {len(downloaded_files)} DICOM files")
    os.makedirs(IMAGES_PATH, exist_ok=True)
    
    converted_count = 0
    
    for dicom_path in tqdm(downloaded_files, desc="Converting"):
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
        
        if convert_dicom_to_image(dicom_path, output_path):
            converted_count += 1
    
    print(f"✅ Converted {converted_count}/{len(downloaded_files)} files")
    print(f"📂 Images: {IMAGES_PATH}")
    
    if SAVE_MASKS:
        print(f"🎯 V1.3 style overlays: {MASKS_PATH}")
    
    print("🎉 V1.4 Simplified processing complete!")
else:
    print("⚠️ No files to convert or conversion disabled")


**<h1 align="center">Download ArchiMed Images V1.4 - SIMPLIFIED</h1>**

## **V1.4: Simplified Lung Segmentation & Cropping**
- **Single Model**: Uses best available segmentation model (TorchXRayVision preferred)
- **Clear Area Cropping**: Crops to the clear rectangular area (matches overlay bright zone)
- **Simple Output**: Basic mask and overlay generation
- **Minimal Configuration**: Only essential parameters

## **Key Simplifications from V1.3:**
- Removed ensemble methods and multiple thresholds
- Removed enhanced preprocessing options
- Simplified overlay generation
- Reduced configuration parameters
- Cleaner, more focused code

In [ ]:
# Simple Configuration
CSV_FOLDER = "/home/pyuser/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"

# Paths
DOWNLOAD_PATH = '/home/pyuser/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'

# Processing settings
TARGET_SIZE = (518, 518)
LUNG_THRESHOLD = 0.1  # Single threshold for lung detection
CROP_MARGIN = 40  # Margin around lungs for final crop

# Options
CONVERT = True
SAVE_MASKS = True

print("V1.4 Simplified configuration loaded!")
print(f"Target size: {TARGET_SIZE}")
print(f"Lung threshold: {LUNG_THRESHOLD}")

In [ ]:
# Core dependencies
import ArchiMedConnector.A3_Connector as A3_Conn
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
import cv2
import subprocess
import sys

print("Core dependencies loaded")

# Initialize ArchiMed connector
a3conn = A3_Conn.A3_Connector()

In [ ]:
# Simple segmentation model setup
segmentation_model = None
model_type = None

# Try TorchXRayVision first (best option)
try:
    import torchxrayvision as xrv
    import torch
    segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
    model_type = 'torchxray'
    print("✅ TorchXRayVision loaded")
except ImportError:
    print("⚠️ Installing TorchXRayVision...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torchxrayvision"])
        import torchxrayvision as xrv
        import torch
        segmentation_model = xrv.baseline_models.chestx_det.PSPNet()
        model_type = 'torchxray'
        print("✅ TorchXRayVision installed and loaded")
    except:
        print("❌ TorchXRayVision unavailable, using fallback")
        model_type = 'fallback'

print(f"Segmentation method: {model_type}")

In [ ]:
def simple_lung_segmentation(image):
    """Simple lung segmentation using available model"""
    
    if model_type == 'torchxray':
        # TorchXRayVision segmentation
        if len(image.shape) == 3:
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            image_gray = image
        
        # Normalize and resize for model
        image_norm = xrv.datasets.normalize(image_gray, 255)
        image_norm = image_norm[None, ...]  
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        # Run inference
        with torch.no_grad():
            output = segmentation_model(image_tensor)
        
        # Extract lung masks
        lung_mask = np.zeros((512, 512))
        for i, target in enumerate(segmentation_model.targets):
            if target in ['Left Lung', 'Right Lung']:
                lung_mask += output[0, i].cpu().numpy()
        
        # Resize back and threshold
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        binary_mask = (lung_mask > LUNG_THRESHOLD).astype(np.uint8)
        
    else:
        # Fallback segmentation
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # Simple thresholding approach
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        gray = clahe.apply(gray)
        
        _, otsu_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        adaptive_mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        combined = np.maximum(otsu_mask * 0.7, adaptive_mask * 0.3)
        binary_mask = (combined > 128).astype(np.uint8)
    
    # Clean up mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
    
    return binary_mask

def crop_to_clear_area(image, binary_mask):
    """Crop image to clear area (like overlay bright zone)"""
    
    # Get lung boundaries
    coords = np.column_stack(np.where(binary_mask > 0))
    if len(coords) == 0:
        return image  # No lungs found, return original
    
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    
    # Calculate clear area boundaries (like overlay)
    h, w = image.shape[:2]
    lung_center_y = (y_min + y_max) // 2
    lung_center_x = (x_min + x_max) // 2
    
    # Calculate crop size with target aspect ratio
    target_aspect = TARGET_SIZE[0] / TARGET_SIZE[1]
    lung_width = x_max - x_min + 2 * CROP_MARGIN
    lung_height = y_max - y_min + 2 * CROP_MARGIN
    
    if lung_width / lung_height > target_aspect:
        crop_width = lung_width
        crop_height = int(crop_width / target_aspect)
    else:
        crop_height = lung_height
        crop_width = int(crop_height * target_aspect)
    
    # Center crop on lungs
    crop_x_min = max(0, lung_center_x - crop_width // 2)
    crop_y_min = max(0, lung_center_y - crop_height // 2)
    crop_x_max = min(w, crop_x_min + crop_width)
    crop_y_max = min(h, crop_y_min + crop_height)
    
    # Adjust if hitting boundaries
    if crop_x_max == w:
        crop_x_min = w - crop_width
    if crop_y_max == h:
        crop_y_min = h - crop_height
    
    crop_x_min = max(0, crop_x_min)
    crop_y_min = max(0, crop_y_min)
    
    # Crop image
    if len(image.shape) == 3:
        cropped = image[crop_y_min:crop_y_max, crop_x_min:crop_x_max, :]
    else:
        cropped = image[crop_y_min:crop_y_max, crop_x_min:crop_x_max]
    
    return cropped, (crop_x_min, crop_y_min, crop_x_max, crop_y_max)

def save_simple_masks(image, binary_mask, crop_bounds, file_id):
    """Save simple mask and overlay"""
    if not SAVE_MASKS:
        return
    
    os.makedirs(MASKS_PATH, exist_ok=True)
    
    # Save binary mask
    mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
    mask_image = (binary_mask * 255).astype(np.uint8)
    cv2.imwrite(mask_path, mask_image)
    
    # Save simple overlay
    overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
    overlay = image.copy()
    if len(overlay.shape) == 2:
        overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
    
    # Darken areas outside crop
    crop_x_min, crop_y_min, crop_x_max, crop_y_max = crop_bounds
    mask = np.ones(overlay.shape[:2], dtype=bool)
    mask[crop_y_min:crop_y_max, crop_x_min:crop_x_max] = False
    overlay[mask] = (overlay[mask] * 0.5).astype(np.uint8)
    
    # Add lung contours in red
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(overlay, contours, -1, (0, 0, 255), 2)
    
    # Add crop rectangle in cyan
    cv2.rectangle(overlay, (crop_x_min, crop_y_min), (crop_x_max, crop_y_max), (255, 255, 0), 2)
    
    cv2.imwrite(overlay_path, overlay)

print("Simple processing functions loaded")

In [ ]:
# Load CSV and download files
try:
    user_info = a3conn.getUserInfos()
    print(f"ArchiMed user: {user_info}")
    
    # Load CSV
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    print(f"Loaded CSV with {len(df)} rows")
    
    # Find FileID column
    file_id_column = None
    for col in ['FileID', 'file_id', 'File_ID']:
        if col in df.columns:
            file_id_column = col
            break
    
    if file_id_column is None:
        raise ValueError("FileID column not found")
    
    # Get file IDs
    file_ids = df[file_id_column].dropna().unique()
    total_files = len(file_ids)
    
    print(f"Starting download of {total_files} files")
    
    # Download files
    downloaded_files = []
    os.makedirs(DOWNLOAD_PATH, exist_ok=True)
    
    for i, file_id in enumerate(file_ids):
        progress = ((i + 1) / total_files) * 100
        file_id_str = str(file_id)
        print(f"Downloading {file_id_str} ({progress:.1f}% - {i+1}/{total_files})")
        
        dicom_file_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
        
        if os.path.exists(dicom_file_path):
            print(f"File {file_id} already exists, skipping")
            downloaded_files.append(dicom_file_path)
            continue
        
        try:
            result = a3conn.downloadFile(
                int(file_id_str),
                asStream=False,
                destDir=DOWNLOAD_PATH,
                filename=f"{file_id_str}.dcm",
                inWorklist=False
            )
            
            if result and os.path.exists(dicom_file_path):
                downloaded_files.append(dicom_file_path)
                print(f"✅ Downloaded: {dicom_file_path}")
            else:
                print(f"⚠️ Download unclear for {file_id_str}")
        except Exception as e:
            print(f"⚠️ Failed to download {file_id_str}: {e}")
    
    print(f"Downloaded {len(downloaded_files)} files successfully")
    
except Exception as e:
    print(f"Setup failed: {e}")
    downloaded_files = []

In [ ]:
# Convert DICOM files with simplified processing
def convert_dicom_simple(dicom_path, output_path):
    """Simple DICOM conversion with lung segmentation and clear area cropping"""
    try:
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        
        # Read DICOM
        dicom_data = pydicom.dcmread(dicom_path)
        image_array = dicom_data.pixel_array
        
        # Handle photometric interpretation
        if hasattr(dicom_data, 'PhotometricInterpretation'):
            if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                image_array = np.max(image_array) - image_array
        
        # Normalize to 0-255
        if image_array.max() > 255:
            image_array = ((image_array - image_array.min()) / 
                          (image_array.max() - image_array.min()) * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)
        
        print(f"Processing {file_id}: {image_array.shape}")
        
        # Lung segmentation
        binary_mask = simple_lung_segmentation(image_array)
        
        # Check if segmentation found reasonable lung area
        lung_ratio = np.sum(binary_mask) / binary_mask.size
        if lung_ratio < 0.01 or lung_ratio > 0.8:
            print(f"⚠️ Unusual lung ratio {lung_ratio:.3f}, using original image")
            processed_image = image_array
            crop_bounds = (0, 0, image_array.shape[1], image_array.shape[0])
        else:
            # Crop to clear area
            processed_image, crop_bounds = crop_to_clear_area(image_array, binary_mask)
            print(f"✅ Cropped to clear area: {processed_image.shape}")
        
        # Save masks
        save_simple_masks(image_array, binary_mask, crop_bounds, file_id)
        
        # Convert to PIL and resize
        if len(processed_image.shape) == 2:
            pil_image = Image.fromarray(processed_image, mode='L')
        else:
            pil_image = Image.fromarray(processed_image)
        
        # Maintain aspect ratio by cropping then resizing
        current_width, current_height = pil_image.size
        target_width, target_height = TARGET_SIZE
        
        current_ratio = current_width / current_height
        target_ratio = target_width / target_height
        
        if current_ratio > target_ratio:
            # Crop width
            new_width = int(current_height * target_ratio)
            left = (current_width - new_width) // 2
            pil_image = pil_image.crop((left, 0, left + new_width, current_height))
        else:
            # Crop height
            new_height = int(current_width / target_ratio)
            top = (current_height - new_height) // 2
            pil_image = pil_image.crop((0, top, current_width, top + new_height))
        
        # Final resize
        pil_image = pil_image.resize(TARGET_SIZE, Image.Resampling.LANCZOS)
        
        # Save
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        pil_image.save(output_path)
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to convert {dicom_path}: {e}")
        return False

# Convert all downloaded files
if CONVERT and downloaded_files:
    print(f"Converting {len(downloaded_files)} DICOM files")
    
    os.makedirs(IMAGES_PATH, exist_ok=True)
    converted_count = 0
    
    for dicom_path in tqdm(downloaded_files, desc="Converting"):
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
        
        if convert_dicom_simple(dicom_path, output_path):
            converted_count += 1
    
    print(f"✅ Successfully converted {converted_count}/{len(downloaded_files)} files")
    print(f"Images saved to: {IMAGES_PATH}")
    if SAVE_MASKS:
        print(f"Masks saved to: {MASKS_PATH}")
    print("🎉 V1.4 Simplified processing complete!")
else:
    print("⚠️ No files to convert or conversion disabled")